In [19]:
import json
from random import seed, shuffle
from time import perf_counter
from typing import Literal, TypedDict

import numpy as np
from astropy.timeseries import LombScargle
from constants import DATA_DIR
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from xgboost import XGBRegressor

from astrofit.model import Asteroid, Lightcurve, LightcurveBin
from astrofit.utils import (
    AsteroidLoader,
    LightcurveBinner,
    LightcurvePlotter,
    LightcurveSplitter,
)

In [2]:
asteroid_loader = AsteroidLoader(DATA_DIR)
lightcurve_binner = LightcurveBinner()
lightcurve_plotter = LightcurvePlotter()
lightcurve_splitter = LightcurveSplitter()

ASTEROIDS_FREQ_DATA_PATH = DATA_DIR / "asteroids_freq_data.json"

In [3]:
MAX_PERIOD = 13

asteroids: list[Asteroid] = []
for asteroid_name in tqdm(asteroid_loader.available_asteroids):
    asteroid = asteroid_loader.load_asteroid(asteroid_name)
    if asteroid.period > MAX_PERIOD:
        continue

    asteroids.append(asteroid)

print(f"Loaded {len(asteroids)} asteroids")

100%|██████████| 329/329 [00:02<00:00, 111.74it/s]

Loaded 203 asteroids


In [4]:
class Config(TypedDict):
    max_hours_diff: float
    min_no_points: int
    top_k_bins: int
    select_bins_by: Literal["lightcurves", "points"]
    max_time_diff: float
    min_bin_size: int
    top_k_freqs: int
    nterms: int


In [5]:
def _split_lightcurves(asteroid: Asteroid, config: Config, debug: bool) -> list[Lightcurve]:
    max_hours_diff = config["max_hours_diff"]
    min_no_points = config["min_no_points"]

    if debug:
        print(
            f"Splitting lightcurves for {asteroid.name} with max_hours_diff={max_hours_diff} and min_no_points={min_no_points}"
        )

        print(f"Before splitting: {len(asteroid.lightcurves)} lightcurves")

    splitted = lightcurve_splitter.split_lightcurves(
        asteroid.lightcurves,
        max_hours_diff=max_hours_diff,
        min_no_points=min_no_points,
    )

    if debug:
        print(f"After splitting: {len(splitted)} lightcurves")

    return splitted


def _get_top_k_bins(lightcurves: list[Lightcurve], config: Config, debug: bool) -> list[LightcurveBin]:
    max_time_diff = config["max_time_diff"]
    min_bin_size = config["min_bin_size"]
    top_k_bins = config["top_k_bins"]

    if debug:
        print(f"Getting top {top_k_bins} bins with max_time_diff={max_time_diff} and min_bin_size={min_bin_size}")

    bins = lightcurve_binner.bin_lightcurves(
        lightcurves,
        max_time_diff=max_time_diff,
        min_bin_size=min_bin_size,
    )

    if debug:
        print(f"After binning {len(bins)} bins available")

    if len(bins) < top_k_bins:
        print(f"WARNING - using {len(bins)} bins instead of {top_k_bins}")

    if config["select_bins_by"] == "lightcurves":
        return sorted(bins, reverse=True)[:top_k_bins]

    elif config["select_bins_by"] == "points":
        return sorted(bins, key=lambda bin: bin.points_count, reverse=True)[:top_k_bins]

    else:
        raise ValueError("Invalid value for select_bins_by")


def _get_top_k_freqs(lightcurve_bin: LightcurveBin, config: Config, debug: bool):
    nterms = config["nterms"]
    top_k_freqs = config["top_k_freqs"]

    if debug:
        print(
            f"Getting top {top_k_freqs} frequencies with nterms={nterms} for "
            f"lightcurves={len(lightcurve_bin)} with total points={len(lightcurve_bin.times)}"
        )

    frequency, power = LombScargle(
        lightcurve_bin.times,
        lightcurve_bin.brightnesses,
        nterms=nterms,
    ).autopower(maximum_frequency=400)

    idx = np.argsort(power)[::-1][:top_k_freqs]
    top_k_freqs = frequency[idx]

    return np.sort(top_k_freqs)


def get_freq_features(asteroid: Asteroid, config: Config, as_list: bool = False, debug: bool = False) -> list | np.ndarray | None:
    splitted_lightcurves = _split_lightcurves(asteroid, config, debug)
    top_k_bins = _get_top_k_bins(splitted_lightcurves, config, debug)

    if not top_k_bins:  #  or len(top_k_bins) < config["top_k_bins"]:
        if debug:
            print(f"{'-'*50}\n")

        return

    freq_features = []
    for _bin in top_k_bins:
        top_k_freqs = _get_top_k_freqs(_bin, config, debug)
        freq_features.extend(top_k_freqs)

    if debug:
        print(f"{'-'*50}\n")

    if as_list:
        return freq_features

    return np.array(freq_features)

In [6]:
config = Config(
    max_hours_diff=24,
    min_no_points=10,
    top_k_bins=4,
    select_bins_by="points",
    max_time_diff=45,
    min_bin_size=1,
    top_k_freqs=50,
    nterms=3,
)

In [7]:
failed_cnt = 0
asteroids_data = {}
for asteroid in tqdm(asteroids):
    start = perf_counter()
    freq_features = get_freq_features(asteroid, config, as_list=True, debug=True)
    processing_time = perf_counter() - start

    asteroids_data[asteroid.name] = {
        "is_failed": False,
        "period": asteroid.period,
        "processing_time": processing_time,
        "freq_features": [],
    }
    if freq_features is None:
        failed_cnt += 1
        asteroids_data[asteroid.name]["is_failed"] = True
        continue

    asteroids_data[asteroid.name]["freq_features"] = freq_features

print(f"Failed asteroids: {failed_cnt} ({failed_cnt / len(asteroids) * 100:.2f}%)")
print(f"{'-'*50}")


with open(ASTEROIDS_FREQ_DATA_PATH, "w") as f:
    json.dump(asteroids_data, f, indent=4)

  0%|          | 0/203 [00:00<?, ?it/s]

Splitting lightcurves for Abundantia with max_hours_diff=24 and min_no_points=10
Before splitting: 1 lightcurves
After splitting: 5 lightcurves
Getting top 4 bins with max_time_diff=45 and min_bin_size=1
After binning 2 bins available
WARNING - using 2 bins instead of 4
Getting top 50 frequencies with nterms=3 for lightcurves=2 with total points=37


  0%|          | 1/203 [00:02<09:42,  2.88s/it]

Getting top 50 frequencies with nterms=3 for lightcurves=1 with total points=10
--------------------------------------------------

Splitting lightcurves for Adelheid with max_hours_diff=24 and min_no_points=10
Before splitting: 31 lightcurves
After splitting: 31 lightcurves
Getting top 4 bins with max_time_diff=45 and min_bin_size=1
After binning 7 bins available
Getting top 50 frequencies with nterms=3 for lightcurves=8 with total points=377
Getting top 50 frequencies with nterms=3 for lightcurves=5 with total points=167
Getting top 50 frequencies with nterms=3 for lightcurves=4 with total points=130
Getting top 50 frequencies with nterms=3 for lightcurves=4 with total points=126


  1%|          | 2/203 [00:13<24:43,  7.38s/it]

...

--------------------------------------------------

Splitting lightcurves for Victoria with max_hours_diff=24 and min_no_points=10
Before splitting: 38 lightcurves
After splitting: 44 lightcurves
Getting top 4 bins with max_time_diff=45 and min_bin_size=1
After binning 9 bins available
Getting top 50 frequencies with nterms=3 for lightcurves=4 with total points=1109
Getting top 50 frequencies with nterms=3 for lightcurves=6 with total points=925
Getting top 50 frequencies with nterms=3 for lightcurves=8 with total points=532
Getting top 50 frequencies with nterms=3 for lightcurves=6 with total points=477


100%|█████████▉| 202/203 [18:56<00:06,  6.54s/it]

--------------------------------------------------

Splitting lightcurves for Weringia with max_hours_diff=24 and min_no_points=10
Before splitting: 10 lightcurves
After splitting: 19 lightcurves
Getting top 4 bins with max_time_diff=45 and min_bin_size=1
After binning 6 bins available
Getting top 50 frequencies with nterms=3 for lightcurves=6 with total points=222
Getting top 50 frequencies with nterms=3 for lightcurves=2 with total points=150


100%|██████████| 203/203 [19:01<00:00,  5.62s/it]

Getting top 50 frequencies with nterms=3 for lightcurves=1 with total points=54
Getting top 50 frequencies with nterms=3 for lightcurves=1 with total points=20
--------------------------------------------------

Failed asteroids: 40 (19.70%)
--------------------------------------------------


In [9]:
with open(ASTEROIDS_FREQ_DATA_PATH, "w") as f_out:
    json.dump(asteroids_data, f_out, indent=4)

In [12]:
with open(ASTEROIDS_FREQ_DATA_PATH, "r") as f_in:
    asteroids_data = json.load(f_in)


In [18]:
filtered_data = {name: data for name, data in asteroids_data.items() if not data["is_failed"]}
len(filtered_data)

163

In [23]:
X = [data["freq_features"] for data in filtered_data.values()]
y = np.array([data["period"] for data in filtered_data.values()])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
space = {
    "max_depth": hp.quniform("max_depth", 3, 18, 1),
    "gamma": hp.uniform("gamma", 1, 9),
    "reg_alpha": hp.quniform("reg_alpha", 1, 100, 1),
    "reg_lambda": hp.quniform("reg_lambda", 1, 100, 1),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
    "min_child_weight": hp.quniform("min_child_weight", 1, 10, 1),
    "n_estimators": hp.quniform("n_estimators", 100, 2000, 1),
    "eta": hp.uniform("eta", 0.01, 0.3),
    "tree_method": hp.choice("tree_method", ["auto", "exact", "approx", "hist"]),
    "max_leaves": hp.quniform("max_leaves", 2, 2**10, 1),
    "max_bin": hp.quniform("max_bin", 2, 2**10, 1),
    "num_parallel_tree": hp.quniform("num_parallel_tree", 1, 10, 1),
    "early_stopping_rounds": hp.quniform("early_stopping_rounds", 10, 100, 1),
    "seed": 88,
}

In [ ]:
def objective(space: dict):
    model = XGBRegressor(
        max_depth=int(space["max_depth"]),
        gamma=space["gamma"],      
        reg_alpha=int(space["reg_alpha"]),
        reg_lambda=space["reg_lambda"],
        colsample_bytree=space["colsample_bytree"],
        min_child_weight=int(space["min_child_weight"]),
        n_estimators=int(space["n_estimators"]),
        eta=space["eta"],
        tree_method=space["tree_method"],
        max_leaves=int(space["max_leaves"]),
        max_bin=int(space["max_bin"]),
        num_parallel_tree=int(space["num_parallel_tree"]),
        eval_metric=mean_squared_error,
        early_stopping_rounds=int(space["early_stopping_rounds"]),
        n_jobs=-1,
    )

    evaluation = [(train_X, train_y), (test_X, test_y)]
    model.fit(train_X, train_y, eval_set=evaluation, verbose=False)

    pred_y = model.predict(test_X)
    mse = mean_squared_error(test_y, pred_y)

    return {"loss": mse, "status": STATUS_OK}

In [ ]:
trials = Trials()

best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=1000,
    trials=trials,
)

In [ ]:
best

In [ ]:
model = XGBRegressor(
    max_depth=int(best["max_depth"]),
    gamma=best["gamma"],      
    reg_alpha=int(best["reg_alpha"]),
    reg_lambda=best["reg_lambda"],
    colsample_bytree=best["colsample_bytree"],
    min_child_weight=int(best["min_child_weight"]),
    n_estimators=int(best["n_estimators"]),
    eta=best["eta"],
    tree_method="exact",
    max_leaves=int(best["max_leaves"]),
    max_bin=int(best["max_bin"]),
    num_parallel_tree=int(best["num_parallel_tree"]),
    eval_metric=mean_squared_error,
    early_stopping_rounds=int(best["early_stopping_rounds"]),
    n_jobs=-1,
)

evaluation = [(train_X, train_y), (test_X, test_y)]
model.fit(train_X, train_y, eval_set=evaluation, verbose=False)

pred_y = model.predict(test_X)
mse = mean_squared_error(test_y, pred_y)

print(f"Mean Squared Error: {mse}\n")

for pred, true in zip(pred_y, test_y):
    print(f"Predicted: {pred:.2f}, True: {true:.2f}")

In [ ]:
[7.51 - 8.33, 9.60 - 9.93, 9.87 - 10.74, 7.61 - 7.28, 11.68 - 11.66, 5.75 - 5.22]